In [3]:
import pandas as pd
from sqlalchemy import create_engine, Table, MetaData
from sqlalchemy.types import String, Integer, Float, DateTime
import os

In [4]:
# Función para detectar el tipo de dato de pandas y convertirlo en un tipo SQL compatible
def detect_sqlalchemy_type(col_dtype):
    if pd.api.types.is_integer_dtype(col_dtype):
        return Integer()
    elif pd.api.types.is_float_dtype(col_dtype):
        return Float()
    elif pd.api.types.is_datetime64_any_dtype(col_dtype):
        return DateTime()
    else:
        return String()  # Cualquier otro tipo será tratado como texto

In [5]:
# Función para crear una tabla con el esquema detectado
def create_table_from_dataframe(df, table_name, engine):
    metadata = MetaData()
    columns = []
    for col_name, col_dtype in df.dtypes.items():
        col_type = detect_sqlalchemy_type(col_dtype)
        columns.append((col_name, col_type))
    
    # Crear la tabla en la base de datos
    table = Table(
        table_name,
        metadata,
        *[Column(col_name, col_type) for col_name, col_type in columns]
    )
    
    metadata.create_all(engine)

In [ ]:
def main():
    # Conexión a la base de datos PostgreSQL
    user = 'postgres'
    password = 'postgres'
    host = 'postgres'
    port = '5432'
    database = 'dw'
    
    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
    
    # Ruta del archivo a leer (puede ser CSV o Excel)
    file_path = './data/experiencias_por_pais.csv'  # O .xlsx para Excel
    
    # Detectar el formato y cargar el archivo
    file_ext = os.path.splitext(file_path)[1]
    if file_ext == '.csv':
        df = pd.read_csv(file_path)
    elif file_ext in ['.xls', '.xlsx']:
        df = pd.read_excel(file_path)
    else:
        raise ValueError(f'Formato de archivo no soportado: {file_ext}')
    
    # Nombre de la tabla en la base de datos
    table_name = 'staging.nombre_tabla_en_postgres'
    
    # Crear la tabla en PostgreSQL con el esquema detectado
    create_table_from_dataframe(df, table_name, engine)
    
    # Insertar los datos en la tabla
    df.to_sql(table_name, engine, if_exists='replace', index=False)

    print(f'Tabla "{table_name}" creada y datos insertados correctamente.')

In [ ]:
if __name__ == '__main__':
    main()